In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import pymysql


from sqlalchemy import create_engine
from tqdm import tqdm
from binance import Client 
sns.set()
pymysql.install_as_MySQLdb()


In [ ]:
engine = create_engine('sqlite:///Binance_daily.db')

In [ ]:
client = Client()

In [ ]:
symbols = ['BTCUSDT','ETHUSDT', 'ADAUSDT', 'BNBUSDT']

In [ ]:
def get_data(symbol, interval = client.KLINE_INTERVAL_12HOUR, start = '2018-06-01'):
    temp = client.get_historical_klines(symbol, interval, start)
    temp = pd.DataFrame(temp)
    temp = temp.iloc[:,:6]
    temp.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    temp.Date = pd.to_datetime(temp.Date, unit = "ms")
    temp = temp.set_index("Date", drop = True)
    temp = temp.astype(float)
    return temp

In [ ]:
for coin in tqdm(symbols):
    df = get_data(coin)
    df.to_sql(coin, if_exists= 'replace', con = engine)

In [ ]:
def qry(symbol):
    df = pd.read_sql(symbol, engine)
    df = df.set_index('Date')
    df = df.asfreq('12H')
    return df.Close

In [ ]:
def apply_techinical(df):
    
    for column in df.columns.to_list():
        #calculating returns
        df[f'Ret_{column}'] = df[column].pct_change().mul(100)
        df[f'Ret_cum_{column}'] = df[f'Ret_{column}'].cumsum()
        #normalizing values
        benchmark = df.loc[:,column][0]
        df[f'Norm_{column}'] = df[column].div(benchmark).mul(100)
    return df
        

In [ ]:
def process_coin(symbols):
    df_list = []
    
    for coin in symbols:
        df = qry(coin)
        df.name = coin 
        df_list.append(df)
    df = pd.DataFrame(df_list).T
    df = apply_techinical(df)
    
    return df

In [ ]:
df = process_coin(symbols = symbols)

In [ ]:
df.head()

In [ ]:
def train_split(df, size:int):
    percentage = int(len(df) * size)
    data = df.iloc[:percentage]
    data_test = df.iloc[percentage:]
    
    data.to_csv('../data/processed/data_processed.csv')
    data_test.to_csv('../data/processed/data_test.csv')
    
    return len(data), len(data_test)

In [ ]:
train_split(df, 0.7)